<a href="https://colab.research.google.com/github/MonishSoundarRaj/data_check_generator/blob/main/data_preprocessing_for_genertor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data_theta.csv")

In [3]:
df.head(10)

,Unnamed: 0,job,user,project,state,gpu_num,cpu_num,node_num,submit_time,wait_time,run_time,wall_time,node_hour,new_status
0,0,639487,0.000000e+00,0.000000e+00,0,0,512.0,8.0,1672529301,71,3632.92,3600.0,8.073156,Killed
1,1,639448,0.000000e+00,0.000000e+00,0,0,16384.0,256.0,1672507846,7772,21654.66,21600.0,1539.886933,Pass
2,2,639135,0.000000e+00,0.000000e+00,0,0,262144.0,4096.0,1672174846,340972,21655.97,21600.0,24639.681422,Killed
3,3,638720,7.029633e+13,7.029633e+13,0,0,262144.0,4096.0,1671905072,632522,3648.70,3600.0,4151.409778,Killed
4,4,639463,0.000000e+00,0.000000e+00,0,0,8192.0,128.0,1672513950,23424,5995.18,10800.0,213.161956,Pass
5,5,638722,7.029633e+13,7.029633e+13,0,0,262144.0,4096.0,1671905120,636224,3634.13,3600.0,4134.832356,Killed
6,6,638723,7.029633e+13,7.029633e+13,0,0,262144.0,4096.0,1671905152,639947,3669.52,3600.0,4175.098311,Killed
7,7,639488,0.000000e+00,0.000000e+00,0,0,8192.0,128.0,1672543325,45,5880.94,10800.0,209.100089,Pass
8,8,639489,0.000000e+00,0.000000e+00,0,0,8192.0,128.0,1672549223,66,5829.52,10800.0,207.271822,Pass
9,9,639490,0.000000e+00,0.000000e+00,0,0,8192.0,128.0,1672555053,46,5235.68,10800.0,186.157511,Pass


In [4]:
df.sort_values(by="submit_time", inplace=True)
df.reset_index(inplace=True, drop=True)
df.head(10)

,Unnamed: 0,job,user,project,state,gpu_num,cpu_num,node_num,submit_time,wait_time,run_time,wall_time,node_hour,new_status
0,109,632190,0.000000e+00,0.000000e+00,0,0,32768.0,512.0,1668693697,3962980,18431.92,21600.0,2621.428622,Pass
1,117,634597,0.000000e+00,0.000000e+00,0,0,49152.0,768.0,1669819930,2844400,18341.22,21600.0,3912.793600,Pass
2,83,636183,3.507588e+13,3.507588e+13,0,0,16384.0,256.0,1670608094,2049318,2854.59,21600.0,202.993067,Pass
3,50,636192,3.507588e+13,3.507588e+13,0,0,16896.0,264.0,1670609219,2025910,7644.15,17820.0,560.571000,Pass
4,41,637840,7.897148e+13,7.897148e+13,0,0,40960.0,640.0,1671471792,1120443,43269.53,43200.0,7692.360889,Killed
5,17,637860,0.000000e+00,0.000000e+00,0,0,163840.0,2560.0,1671479504,1069365,42388.64,46800.0,30143.032889,Pass
6,112,637917,0.000000e+00,0.000000e+00,0,0,57344.0,896.0,1671509425,1081954,86477.70,86400.0,21523.338667,Killed
7,18,638453,0.000000e+00,0.000000e+00,0,0,40960.0,640.0,1671747802,801087,43245.01,43200.0,7688.001778,Killed
8,226,638563,3.459062e+13,3.459062e+13,0,0,203520.0,3180.0,1671810411,937656,1755.37,2400.0,1550.576833,Pass
9,38,638597,0.000000e+00,0.000000e+00,0,0,41472.0,648.0,1671829841,761519,43229.14,43200.0,7781.245200,Killed


In [5]:
if 'new_status' not in df.columns:
  df.rename(columns={'state': 'new_status'}, inplace=True)

In [6]:
df['interval'] = df['submit_time'] - df['submit_time'].shift(1)
df['interval'] = df['interval'].fillna(0)

In [7]:
negative_submit_times_df = df['interval'] < 0
print(f"Number of negative submit times in df: {negative_submit_times_df.sum()}")

Number of negative submit times in df: 0


In [8]:
df = df.head(15000)

In [9]:
print(df['job'].nunique())
print(df['user'].nunique())
print(df['project'].nunique())

14374
47
47


In [10]:
unique_values = {}
for col in ['job', 'user', 'project']:
  unique_values[col] = {}
  unique_values[col]['values'] = df[col].unique()
  unique_values[col]['mapping'] = dict(zip(unique_values[col]['values'], range(1, 15001)))

for i, row in df.iterrows():
  for col in ['job', 'user', 'project']:
    df.at[i, col] = unique_values[col]['mapping'][row[col]]

In [11]:
df['job'] = df['job'].astype(int)
df['user'] = df['user'].astype(int)
df['project'] = df['project'].astype(int)

In [12]:
print(df['job'].max())
print(df['user'].max())
print(df['project'].max())

14374
47
47


In [13]:
df.head(10)

,Unnamed: 0,job,user,project,state,gpu_num,cpu_num,node_num,submit_time,wait_time,run_time,wall_time,node_hour,new_status,interval
0,109,1,1,1,0,0,32768.0,512.0,1668693697,3962980,18431.92,21600.0,2621.428622,Pass,0.0
1,117,2,1,1,0,0,49152.0,768.0,1669819930,2844400,18341.22,21600.0,3912.793600,Pass,1126233.0
2,83,3,2,2,0,0,16384.0,256.0,1670608094,2049318,2854.59,21600.0,202.993067,Pass,788164.0
3,50,4,2,2,0,0,16896.0,264.0,1670609219,2025910,7644.15,17820.0,560.571000,Pass,1125.0
4,41,5,3,3,0,0,40960.0,640.0,1671471792,1120443,43269.53,43200.0,7692.360889,Killed,862573.0
5,17,6,1,1,0,0,163840.0,2560.0,1671479504,1069365,42388.64,46800.0,30143.032889,Pass,7712.0
6,112,7,1,1,0,0,57344.0,896.0,1671509425,1081954,86477.70,86400.0,21523.338667,Killed,29921.0
7,18,8,1,1,0,0,40960.0,640.0,1671747802,801087,43245.01,43200.0,7688.001778,Killed,238377.0
8,226,9,4,4,0,0,203520.0,3180.0,1671810411,937656,1755.37,2400.0,1550.576833,Pass,62609.0
9,38,10,1,1,0,0,41472.0,648.0,1671829841,761519,43229.14,43200.0,7781.245200,Killed,19430.0


In [14]:
df.rename(columns={'Unnamed: 0': 'u_id'}, inplace=True)

In [15]:
# for philly and helios
# df = df[['u_id','job', 'user', 'project', 'gpu_num', 'cpu_num', 'node_num', 'interval', 'run_time', 'wall_time', 'node_hour', 'core_hour', 'new_status']]
# for bw, and theta
df = df[['u_id','job', 'user', 'project', 'gpu_num', 'cpu_num', 'node_num', 'interval', 'run_time', 'wall_time', 'node_hour', 'new_status']]
df.head()

,u_id,job,user,project,gpu_num,cpu_num,node_num,interval,run_time,wall_time,node_hour,new_status
0,109,1,1,1,0,32768.0,512.0,0.0,18431.92,21600.0,2621.428622,Pass
1,117,2,1,1,0,49152.0,768.0,1126233.0,18341.22,21600.0,3912.793600,Pass
2,83,3,2,2,0,16384.0,256.0,788164.0,2854.59,21600.0,202.993067,Pass
3,50,4,2,2,0,16896.0,264.0,1125.0,7644.15,17820.0,560.571000,Pass
4,41,5,3,3,0,40960.0,640.0,862573.0,43269.53,43200.0,7692.360889,Killed


In [16]:
df.shape

(14374, 12)

In [17]:
df.to_csv("theta_data_training.csv", index=False)

In [18]:
from google.colab import files
files.download("theta_data_training.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>